<a href="https://colab.research.google.com/github/sacamiso/AjedrezDistribuidos/blob/main/practica10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 10.1: Redes recurrentes

En este notebook vamos a presentar en detalle cómo trabajar con distintas versiones de redes recurrentes. Para ello vamos a usar la librería fastai, que ya usamos en el entregable 2, y su librería subyacente que es PyTorch. Este notebook está basado en el libro de [fastai](https://github.com/fastai/fastbook).

Para profundizar en los conceptos de redes recurrentes vamos a construir un modelo de lenguaje desde cero. 


## Instalación librería

Para utilizar este notebook es necesario instalar la versión más actual de la librería de fastai. 

In [1]:
!pip install fastai --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Dataset

Para este notebook vamos a utilizar un dataset llamado *human numbers* que consta de los 10000 primeros números escritos en inglés. Para descargar dicho dataset ejecutamos la siguiente instrucción y creamos una variable path que apunta a donde se ha descargado dicho dataset. 

In [2]:
from fastai.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

Como podemos ver a través de la siguiente instrucción, nuestro dataset consta de dos ficheros de texto plano, uno para entrenar y otro para validar (o testear).

In [3]:
path.ls()

(#2) [Path('/root/.fastai/data/human_numbers/valid.txt'),Path('/root/.fastai/data/human_numbers/train.txt')]

Vamos a combinar ambos dataset y almacenamos el resultado en una variable lines. En el código siguiente aparece un objeto `L()` que es un tipo de lista utilizado en FastAI.

In [4]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

A continuación vamos a formar una única cadena con la lista anterior y a separar cada número mediante un '.'.

In [5]:
text = ' . '.join([l.strip() for l in lines])
text[:100]

'one . two . three . four . five . six . seven . eight . nine . ten . eleven . twelve . thirteen . fo'

Seguidamente aplicamos un proceso de tokenización separando por espacios.

In [6]:
tokens = text.split(' ')
tokens[:10]

['one', '.', 'two', '.', 'three', '.', 'four', '.', 'five', '.']

Como comentamos en teoría, inicialmente la representación de cada palabra suele darse mediante el método one-hot encoding, y se utiliza la red para aprender un embedding de los datos. Sin embargo, la representación one-hot ocupa mucha memoria por lo que en FastAI en lugar de usar dicha representación, se **numericalizan** las palabras. Este proceso consite en representar cada palabra mediante el índice (la posición) que ocupa en el vocabulario. 

Por lo tanto lo primero que tenemos que hacer es construir nuestro vocabulario con las palabras únicas de nuestro dataset. 

In [7]:
vocab = L(*tokens).unique()
vocab

(#30) ['one','.','two','three','four','five','six','seven','eight','nine'...]

Ahora podemos convertir los tokens en números mirando su índice en el vocabulario.

In [8]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#63095) [0,1,2,1,3,1,4,1,5,1...]

Ahora que ya tenemos codificados nuestros datos ya podemos empezar a construir modelos. 

## Un primer modelo

Nuestro objetivo para este dataset va a ser predecir una palabra basándonos en las tres anteriores. Por lo tanto podemos ver un la lista de cada secuencia de tres palabras consecutivas como nuestra "X", y la siguiente palabra de la secuencia como nuestra "y". 

Esto lo podemos lograr en python del siguiente modo. 

In [9]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#21031) [(['one', '.', 'two'], '.'),(['.', 'three', '.'], 'four'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'seven'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'ten'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'thirteen'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'sixteen')...]

Ahora creamos una serie de *tensores* (puede verse como otro tipo de lista) con los valores numéricos, que es lo que el modelo espera como entrada. 

In [10]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#21031) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 7),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 10),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 13),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 16)...]

Ahora podemos preparar una serie de *batches* que serán usados para entrenar la red. Para ello es necesario usar la clase `DataLoader` y separar en un conjunto de entrenamiento y de test. 

In [11]:
# Tamaño del batch
bs = 64
# Partición qu vamos a hacer
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

Ahora vamos a crear una red neuronal que tome tres palabras como entradas y devuelva una predicción para cada palabra del vocabulario indicando la probabilidad de que esa palabra sea la siguiente en la secuencia. En esta red neuronal vamos a tener tres capas completamente conectadas, pero con dos pequeños cambios. 

El primer cambio es que la primera capa solo usará el embedding de la primera palabra como entrada, la segunda capa usará el embedding de la segunda palabra mas la salida de la primera capa como entrada, y la tercera capa usará el embedding de la tercera palabra más la salida de la segunda capa como entrada. El efecto clave de este proceso es que cada palabra se interpreta en el contexto de las palabras precedentes. 

El segundo cambio es que cada una de las capas va usar la misma matriz de pesos. Esto se hace con el objetivo de que el impacto que tiene una palabra en los pesos a partir de las palabras previas no debería cambiar dependiendo de la posición de la palabra. Dicho de otro modo, los los valores de entrada de las capas van cambiando a medida que los datos van pasando a través de las capas, pero los pesos no deben cambiar. De este modo una capa no aprende una posición en la secuencia, sino que tiene que aprender todas ellas. 

Dado que los pesos de la capa no cambian, se puede ver las capas secuenciales como la misma capa repetida. Esto en la práctica se puede llevar a cabo con PyTorch creando una capa y usándola múltiples veces. 

### Nuestro modelo de lenguaje en Pytorch

Vamos a crear el modelo de lenguaje descrito anteriormente en PyTorch.

In [12]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

Cuando queremos crear una red neuronal en PyTorch debemos crear una clase que herede de la clase `Module`. Además debemos proporcionar como se va a calcular la salida a partir de la entrada (es decir la arquitectura de la red) dentro del método `forward`. Además, como vemos en la clase anterior, es normal inicializar una serie de parámetros, en este caso las capas que se van a utilizar, dentro del constructor (método `__init__`) de la clase. 

En este caso tenemos tres capas:

- La capa de embedding (`i_h` por de *input* a *hidden*).
- La capa lineal que crea la salida para la siguiente palabra(`h_h` por de *hidden* a *hidden*).
- Una capa final para predecir la siguiente palabra (`h_o` por de *hidden* a *output*)

Esto puede verse mejor con la siguiente representación gráfica. 

<img alt="Pictorial representation of simple neural network" width="400" src="https://github.com/IA1920/images/blob/master/images/att_00020.png?raw=1" caption="Pictorial representation of simple neural network" id="img_simple_nn">

Cada forma representa una capa: el rectángulo para la entrada, el círculo para la capa oculta, y el triángulo para la capa de salida. Usaremos la misma representación para el resto de diagramas del notebook.

<img alt="Shapes used in our pictorial representations" width="200" src="https://github.com/IA1920/images/blob/master/images/att_00021.png?raw=1" id="img_shapes" caption="Shapes used in our pictorial representations">

Una flecha en el diagrama representa el cálculo realizado, es decir el producto de los pesos por la entrada de la capa seguido de la aplicación de una función de activación. Usando esta notación podemos representar nuestro modelo del siguiente modo. 

<img alt="Representation of our basic language model" width="500" caption="Representation of our basic language model" id="lm_rep" src="https://github.com/IA1920/images/blob/master/images/att_00022.png?raw=1">

Notar que hemos coloreado las flechas de maenra que todas las flechas del mismo color usan la misma matriz de pesos. Por ejemplo, todas las capas de entrada usan la misma matriz de embedding. 

Vamos a entrenar dicho modelo. Para ello debemos constrir un objeto `Learner` que va a recibir 4 parámetros. 
- Un dataloader (`dls`) que indica cómo acceder al dataset.
- La arquitectura de nuestro modelo (`LMModel1`).
- La función de pérdida (`loss_func=F.cross_entropy` que es la función de pérdida utilizada normalmente para clasificación cuando hay múltiples clases). 
- La métrica con la que evaluaremos el modelo (`metrics=accuracy`).

In [13]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)

Ahora ya podemos entrenar nuestro modelo con el método `fit_one_cycle` que vimos en el Entregable 2.

In [14]:
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.876511,2.001736,0.466603,00:01
1,1.406623,1.799482,0.466603,00:01
2,1.420620,1.645612,0.493939,00:01
3,1.388259,1.626691,0.456620,00:01


Para ver qué tal funciona el modelo podemos compararlo con predecir siempre la palabra más común. Para ello vamos a encontrar con el siguiente código cuál es la palabra más común en el conjunto de test.

In [15]:
n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

(tensor(29), 'thousand', 0.15165200855716662)

La palabra más común es "thousand", y usando esta palabra como predicción tendríamos una tasa de acierto del 15%, así que nuestro modelo que obtiene aproximadamente una tasa de acierto del 50% es considerablemente mejor. 

###  Nuestra primera red recurrente

Si nos fijamos en el código de nuestra red podemos notar que hay código que se repite, y que lo podríamos reemplazar usando un bucle. Esto tiene la ventaja de que podremos aplicar nuestra red a secuencias de tokens de distintas longitudes, evitando así estar restringidos a trabajar con secuencias de longitud 3.

In [16]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

Vamos a comprobar que se obtiene los mismos resultados (o muy similares).

In [17]:
learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.835339,1.883517,0.461612,00:02
1,1.401371,1.775067,0.467079,00:01
2,1.419824,1.652131,0.492275,00:01
3,1.395673,1.666135,0.465177,00:01


También podemos refactorizar nuestra representación del mismo modo. 

<img alt="Basic recurrent neural network" width="400" caption="Basic recurrent neural network" id="basic_rnn" src="https://github.com/IA1920/images/blob/master/images/att_00070.png?raw=1">

Ahora que tenemos una red RNN, vamos a mejorarla. 

## Mejorando la RNN

Vamos a ver cómo mejorar la RNN que hemos construido. 

### Manteniendo el estado de una RNN

Si nos fijamos en el código de nuestro modelo, el estado oculto (representando mediante `h`) se reinicia a 0 para cada nueva secuencia. Por lo tanto se pierde toda la información de las secuencias que habían aparecido anteriormente, lo que significa que el modelo no sabe en que punto de la secuencia global se encuentra. Esto se puede resolver facilmente moviendo la inicialización del estado oculto al constructor. 

Sin embargo, esto tiene un problema asociado, debido a que de este modo estaríamos construyendo una red neuronal tan profunda como el número de tokens de nuestro documento. Por ejemplo, si hay 10000 tokens en nuestro dataset, entonces crearíamos una red con 10000 capas. 

Para ver esto, consideremos la imagen original de nuestra red recurrente antes de refactorizarla incluyendo el bucle. En dicha imagen podemos ver que cada capa se corresponde con un token de entrada. 

<img alt="Pictorial representation of simple neural network" width="400" src="https://github.com/IA1920/images/blob/master/images/att_00020.png?raw=1" caption="Pictorial representation of simple neural network" id="img_simple_nn">

El problema con una red de 10000 capas es que cuando llegas a la palabra número diezmil del dataset, todavía necesitas calcular todas las derivadas que van hasta la primera capa. Esto es muy lento, y es bastante improbable que se pueda almacenar un batch en la GPU. 

La solución a este problema consiste en decir a PyTorch que no propague hacia atrás todas las derivadas en la red, y en su lugar se almacenan solo las tres últimas capas de gradientes. Esto se hace mediante el método `detach`.

A continuación tenemos la nueva versión de nuestro modelo RNN. Ahora este modelo almacena el estado ya que recuerda las salidas entre las diferentes llamadas al método `forward`. 

In [18]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

Para usar `LMModel3`, nos tenemos que asegurar que el modelo va a ver las secuencias en un cierto orden. Es decir, el modelo tiene que ver las secuencias en el mismo orden que aparecen en el texto. Para ello tenemos que reorganizar el dataset.

En primer lugar dividimos la muestra en `m = len(dset) // bs` grupos (esto es equivalente a partir el dataset completo en grupos de, por ejemplo, 64 piezas iguales (dado que estamos usando `bs=64`). Por lo tanto, `m` es la longitud de cada una de esas piezas. Por ejemplo, si usamos el dataset completo esto sería:

In [19]:
m = len(seqs)//bs
m,bs,len(seqs)

(328, 64, 21031)

El primer batch estaría compuesto por las muestras:

    (0, m, 2*m, ..., (bs-1)*m)

el segundo por las muestras: 

    (1, m+1, 2*m+1, ..., (bs-1)*m+1)

y así sucesivamente. De este modo, en cada época, el modelo veo una parte continua del texto de tamaño  `3*m` (ya que cada secuencia de texto tiene tamaño 3)

La siguiente función se encarga de ello. 

In [20]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

Seguidamente definimos de nuevo nuestro `DataLoader` pero en este caso le indicamos que descarte el último batch que no tiene la dimensión adecuada (`drop_last=True`) ya que es raro que el tamaño sea divisible exactamente por el `bs`. También le indicamos que el texto se lea en orden mediante `shuffle=False`.

In [21]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

Ahora ya podemos definir nuestro nuevo `Learner` igual que antes con una salvedad. En concreto a nuestro bucle de entrenamiento le vamos a añadir un `Callback`, que son funciones que añaden funcionalidad al proceso de entrenamiento. En este caso el `Callback` va a llamar al método `reset` del modelo al principio de cada época y antes de la fase de validación. Esto hace que podamos empezar con un estado limpio antes de leer nuevos bloques de texto. Además ahora es posible entrenar el modelo por más tiempo sin sufrir overfitting. 

In [22]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.672497,1.787723,0.482692,00:01
1,1.277231,1.809937,0.406971,00:01
2,1.110371,1.684805,0.463221,00:01
3,1.050581,1.770203,0.458413,00:01
4,0.994142,1.695251,0.520673,00:01
5,0.947754,1.613399,0.525240,00:01
6,0.897450,1.549539,0.559856,00:02
7,0.841424,1.583105,0.580048,00:01
8,0.813258,1.651051,0.576683,00:01
9,0.796056,1.652552,0.576442,00:02


### Creando más señal

Otro problema con la aproximación que estamos utilizando es que solo predecimos una palabra para cada tres palabras de entrada. Esto significa que la cantidad de información que le estamos pasando a los pesos cuando se actulizan no es todo lo grande que podría ser. Sería mejor si predijeramos la siguiente palabra a partir de la palabra anterior, en lugar de cada tres palabras como se muestra en la siguiente figura. 

<img alt="RNN predicting after every token" width="400" caption="RNN predicting after every token" id="stateful_rep" src="https://github.com/IA1920/images/blob/master/images/att_00024.png?raw=1">

Esto es bastante sencillo de añadir. Primero tenemos que cambiar nuestros datos de manera que nuestra "y" no sea la palabra que sigue a la última de las tres palabras de la secuencia sino que sea las tres palabras siguientes a cada una de la palabra de la sencuencia. En lugar de 3, vamos a usar un atributo, `ls` (por longitud de secuencia) y a hacerlas un poco más grandes. 

In [23]:
ls = 16
seqs = L((tensor(nums[i:i+ls]), tensor(nums[i+1:i+ls+1]))
         for i in range(0,len(nums)-ls-1,ls))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

Si nos fijamos en el primer elemento de `seqs`, vemos que contiene dos listas del mismo tamaño. La segunda es la misma lista que la primera pero desplazada una posición a la derecha. 

In [24]:
[L(vocab[o] for o in s) for s in seqs[0]]

[(#16) ['one','.','two','.','three','.','four','.','five','.'...],
 (#16) ['.','two','.','three','.','four','.','five','.','six'...]]

Ahora debemos modificar el modelo de manera que realice una predicción cada nueva palabra, en lugar de solo al final de la secuencia. 

In [25]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(ls):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

Este modelo va a devolver salidas de la forma `bs x sl x vocab_sz` (ya que las estamos apilando en `dim=1`). Nuestros objetivos son de la forma `bs x sl`, por lo tanto será necesario aplanarlas antes de calcular la pérdida. Es por esto por lo que definimos una nueva función de pérdida a partir de la `F.cross_entropy`:

In [26]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

Ahora podemos entrenar el modelo. 

In [27]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.228064,3.006139,0.200033,00:00
1,2.303390,1.968454,0.450114,00:00
2,1.751149,1.863843,0.435384,00:00
3,1.458284,1.994506,0.443278,00:00
4,1.265416,1.918844,0.545654,00:00
5,1.116688,2.087994,0.572835,00:00
6,0.992990,2.280013,0.572347,00:00
7,0.902731,2.553467,0.551107,00:00
8,0.831351,2.392303,0.549398,00:00
9,0.774327,2.575213,0.562826,00:00


Es necesario entrenar por más tiempo ya que la tarea es más compleja, y como vemos obtenemos mejores resultados.

Para mejorar el modelo tenemos que aumentar la profundidad del modelo. En este momento nuestra red solo tiene una capa oculta, así que vamos a ver si es posible obtener mejores resultados añadiendo más capas. 

## RNNs multicapa

En una red RNN multicapa, las salidas de nuestra red recurrente se pasan como entrada a una nueva red RNN como se muestra en el siguiente diagrama.

<img alt="2-layer RNN" width="550" caption="2-layer RNN" id="stacked_rnn_rep" src="https://github.com/IA1920/images/blob/master/images/att_00025.png?raw=1">

O si usamos una representación expandida del siguiente modo.

<img alt="2-layer unrolled RNN" width="500" caption="2-layer unrolled RNN" id="unrolled_stack_rep" src="https://github.com/IA1920/images/blob/master/images/att_00026.png?raw=1">

Vamos a ver cómo implementar esto en la práctica.

## El modelo

En lugar de implementar desde cero la red RNN vamos a utilizar la definición de RNN proporcionada por PyTorch que está implementada del mismo modo que hemos explicado anteriormente, pero que además nos da la opción de apilar capas RNN. 

In [28]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

Ahora ya podemos entrenar nuestro nuevo modelo.

In [29]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.037964,2.618287,0.407878,00:01
1,2.157770,1.763017,0.470703,00:01
2,1.702188,1.831890,0.433105,00:00
3,1.480928,1.825001,0.477946,00:00
4,1.315384,1.999529,0.500000,00:00
5,1.164696,2.124962,0.502197,00:00
6,1.036534,2.233096,0.518717,00:00
7,0.924675,2.221634,0.536214,00:00
8,0.831782,2.193921,0.528971,00:00
9,0.763600,2.217889,0.528809,00:00


Como vemos los resultados son peores que con el modelo de una sola capa RNN ¿estamos haciendo algo mal? El problema cuando se aumenta la profundidad de las redes neuronales es que las salidas de las capas tienden o a explotar o a desaparecer, complicando de este modo el proceso de aprendizaje.

### Explosión y desaparición de salidas

En la práctica, crear modelos usando RNN es difícil. Obtendremos mejores resultados si llamamos a la función  `detach` de manera frecuentemente y añadir más capas. De este modo la RNN tiene más tiempo para aprender descriptores más ricos. Pero esto también significa que tenemos un modelo más profundo que entrenar. El problema clave en el desarrollo del deep learning ha sido averiguar como entrenar este tipo de modelos. 

La razón para que esto sea complejo es lo que ocurre cuando multiplicamos una matriz varias veces. Para entender esto pensemos que ocurre cuando multiplicamos un número múltiples veces. Por ejemplo, si multiplicamos por 2 y empezamos por uno tenemos la secuencia 1, 2, 4, 8, ... y después de 32 pasos hemos llegado al número 4294967296. Algo parecido ocurre cuando multiplicamos por 0,5: obtenemos 0.5, 0.25, 0.125, ...  y después de 32 pasos tenemos 0.00000000023. Como podemos ver, un número ligeramente superior o inferior a 1 provoca una explosión o la desaparición de ese número después de unas pocas multiplicaciones.

Debido a que la multiplicación de matrices es solo multiplicar números y sumarlos, el mismo problema aparece al multiplicar matrices. Y una red neuronal al final es una multiplicación repetidad de múltiples matrices. Esto significa que una red neuronal acabe trabajando con números extremadamente grandes o extremadamente pequeños. 

Esto es un problema debido al modo en el que los ordenadores almacenan los números (en "coma flotante"), ya que esta representación se vuelve menos precisa a medida que nos alejamos de cero. Esto se explica de forma clara en el artículo [What you never wanted to know about floating point but will be forced to find out](http://www.volkerschatz.com/science/float.html).

Esto significa que los gradientes que se calculan al actualizar los pesos tienden siendo cero o infinito para las redes profundas. Este problema se conoce como *vanishing gradients* o *exploding gradients*. Esto significa eque en el proceso de propagación hacia atrás, los pesos o bien no se actualizan o saltan al infinito. En cualquiera de los dos casos esto impide el entrenamiento. 

Para resolver este problema, se han diseñado distintas aproximaciones. Una de ellas consiste en cambiar la definición de una capa de forma que se eviten los problemas. Para las RNNs existen dos tipos de capas que evitan estos problemas y son las GRUs, y las LSTM. Ambas están disponibles en PyTorch, pero nosotros nos centraremos en el uso de las LSTM, ya que las GRUs son una pequeña variante de las LSTM. 

## LSTM

Vamos a crear un modelo equivalente al `LMModel5`, usando una LSTM con dos capas.

In [30]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(2, bs, n_hidden) for _ in range(n_layers)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

Pasamos a entrenar el modelo. 

In [31]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.032830,2.770375,0.164551,00:01
1,2.223365,2.010869,0.195638,00:01
2,1.643536,1.712856,0.492839,00:01
3,1.350636,1.742462,0.525553,00:01
4,1.100722,1.852693,0.552490,00:01
5,0.825068,1.371291,0.635905,00:01
6,0.543588,1.438616,0.718424,00:01
7,0.324600,1.240704,0.755290,00:01
8,0.190817,1.144985,0.808512,00:01
9,0.110226,1.222011,0.810628,00:01


En el modelo anterior se puede observar que se produce cierto sobreajuste que se puede resolver aplicando téncicas de regularización.

## Reduciendo el sobreajuste en un LSTM

Las redes recurrentes son díficiles de entrenar debido a los problemas comentado anteriormente. El uso de las LSTM permite entrenar las con más facilidad, pero estas redes tienden a sufrir de sobreajuste. Una técnica para reducir este problema consiste en utilizar *data augmentation* (que no solo se puede aplicar a imágenes sino también a texto). Sin embargo no se suele usar de manera tan habitual en procesado de lenguaje ya que esta técnica suele requerir un modelo que genere aumentos aleatorios (por ejemplo, traduciendo de un lenguaje a otro y volviendo al lenguaje original). 

Sin embargo existen otras técnicas para reducir el sobreajuste en un LSTM que fueron estudiadas de manera detallada en el artículo [Regularizing and Optimizing LSTM Language Models](https://arxiv.org/abs/1708.02182). Este artículo mostró que se pueden mejorar los resultados de las LSTMs aplicando tres técnicas *dropout*, *activation regularization*, y *temporal activation regularization*.

### Dropout

Dropout es una técnica introducida por Geoffrey Hinton et al. en [Improving neural networks by preventing co-adaptation of feature detectors](https://arxiv.org/abs/1207.0580). La idea básica de esta técnica consiste en poner a cero algunas neuronas a medida que se van entrenando. Esto hace que todas las neuronas de la red trabajen para obtener una salida y así evitar neuronas sobre especializadas. Este método se representa con la siguiente figura:

<img src="https://github.com/IA1920/images/blob/master/images/Dropout1.png?raw=1" alt="A figure from the article showing how neurons go off with dropout" width="800" id="img_dropout" caption="A screenshot from the dropout paper">

El autor del artículo explicaba esta técnica con la siguiente metáfora en una entrevista. 

> Fui al banco y me di cuenta que las personas que atendían en el banco cambiaban de manera frecuente. Al preguntar a uno de ellos por la razón de esto me dijo que no lo sabía pero que les movían de puesto de manera frecuente. Descubrí que esto debe deberse a que para defraudar al banco por parte de los empleados es necesario que cooperen. Esto me hizo darme cuenta de que eliminando de forma aleatoria un conjunto diferente de neuronas en cada batch del entrenamiento serviría para prevenir conspiraciones y reducir de este modo el sobreajuste

El uso del dropout no se aplica a todas las capas de la red, sino que es suficiente con aplicarlo justo antes de la última capa de nuestra LSTM para redcuir el sobreajuste. El dropout no solo se utilizan en LSTMs sino que también se puede aplicar en las redes convolucionales o en redes neuronales feed-forward. 

Por último hay que tener en cuenta que el dropout tiene un comportamiento diferente cuando estamos entrenando y cuando estamos usando el modelo ya entrenado. En concreto solo se desactivan neuronas durante el proceso de entrenamiento.

### Las regularizaciones AR y TAR

AR (del inglés *activation regularization*) y TAR (del inglés *temporal activation regularization*) son dos métodos que reducen el sobreajuste incluyendo una penalización a la función de perdida. En concreto, la regularización AR se encarga de que sean las salidas producidas por LSTM lo más pequeñas posibles, mientras que la regularización TAR se encarga de que la salida de dos secuencias consecutivas de nuestro texto sean lo más pequeña posible. 

### Entrenando un modelo con Dropout, AR y TAR

Podemos combinar el Dropout con las regularizaciones AR y TAR para entrenar nuestro modelo. Para ello tenemos que añadir una capa de dropout antes de la salida de nuestro LSTM. Además nuestro modelo debe devolver tres cosas: la salida normal de la red, la salida producida por la capa del dropout, y la salida antes de aplicar el dropout. Las dos últimas serán utilizadas por un callback llamado `RNNRegularization` que se usa para aplicar las regularizaciones AR y TAR.

Otro truco que se suele utilizar en estas redes se conoce como *weight tying*. En un modelo de lenguaje, el embedding representa una función de palabras en inglés (o en otro idioma) a la entrada del modelo, y la salida representa una función de las salidas de la red a palabras en inglés. Podemos esperar que estas funciones sean la misma, esto se puede llevar a cabo asignando la misma matriz de pesos a ambas capas:

    self.h_o.weight = self.i_h.weight

Incluimos estos cambios en el nuevo modelo `LMMModel7`:

In [32]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

Seguidamente creamos nuestro `Learner` usando el callback de `RNNRegularizer` donde `alpha` es un parámetro para la regularización AR, y `beta` se aplica para la regularización TAR. Un `TextLearner` automáticamente añade dichos callbacks por nosotros, así que podemos construir el `Learner` con la siguiente línea:

In [33]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

Ahora podemos entrenar el modelo añadiendo otra técnica de regularización llamada *weight decay* que añade una penalización a la pérdida para que los pesos sean lo más pequeños posibles. 

In [34]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,2.512093,1.839599,0.492025,00:01
1,1.551902,1.352108,0.660726,00:01
2,0.834879,0.981466,0.754232,00:01
3,0.425727,0.799501,0.826497,00:01
4,0.218964,0.770998,0.831787,00:01
5,0.123353,0.661925,0.837158,00:01
6,0.077059,0.674798,0.833496,00:01
7,0.051609,0.622478,0.835531,00:01
8,0.038289,0.627022,0.844401,00:01
9,0.029418,0.565250,0.862142,00:01


Como podemos ver los resultados son bastante mejores a los obtenidos con anterioridad. 

### Ejercicio

A lo largo de este notebook hemos visto cómo crear 7 modelos que a partir de una secuencia de 3 palabras predicen la siguiente palabra del texto. El ejercicio para esta práctica consite en construir y entrenar esos mismos 7 modelos pero prediciendo una palabra a partir de las tres palabras siguientes que aparecen en la secuencia. Añade a continuación tantas celdas como necesites. 

De la siguiente manera obtenemos la secuencia deseada, donde tenemos 3 palabras consecutivas en la X y en la Y la anterior palabra a la secuencia dada.

In [35]:
L((tokens[i:i+3], tokens[i-1]) for i in range(0,len(tokens)-4,3))

(#21031) [(['one', '.', 'two'], 'nine'),(['.', 'three', '.'], 'two'),(['four', '.', 'five'], '.'),(['.', 'six', '.'], 'five'),(['seven', '.', 'eight'], '.'),(['.', 'nine', '.'], 'eight'),(['ten', '.', 'eleven'], '.'),(['.', 'twelve', '.'], 'eleven'),(['thirteen', '.', 'fourteen'], '.'),(['.', 'fifteen', '.'], 'fourteen')...]

Ahora creo una serie de tensores con los valores numéricos, que es lo que el modelo espera como entrada

In [36]:
seqs = L((tensor(nums[i:i+3]), nums[i-1]) for i in range(0,len(nums)-4,3))
seqs

(#21031) [(tensor([0, 1, 2]), 9),(tensor([1, 3, 1]), 2),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 5),(tensor([7, 1, 8]), 1),(tensor([1, 9, 1]), 8),(tensor([10,  1, 11]), 1),(tensor([ 1, 12,  1]), 11),(tensor([13,  1, 14]), 1),(tensor([ 1, 15,  1]), 14)...]

Ahora puedo preparar una serie de batches que serán usados para entrenar la red. Para ello es necesario usar la clase DataLoader y separar en un conjunto de entrenamiento y de test.

In [37]:
# Tamaño del batch
bs = 64
# Partición qu vamos a hacer
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

A continuación paso a crear los modelos:

**Mi modelo de lenguaje en Pytorch**

In [38]:
class LMModel1(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = F.relu(self.h_h(self.i_h(x[:,0])))
        h = h + self.i_h(x[:,1])
        h = F.relu(self.h_h(h))
        h = h + self.i_h(x[:,2])
        h = F.relu(self.h_h(h))
        return self.h_o(h)

In [39]:
learn = Learner(dls, LMModel1(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)

In [40]:
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.859495,2.156695,0.465652,00:01
1,1.366194,1.791498,0.470406,00:01
2,1.400366,1.557071,0.495127,00:01
3,1.337824,1.588358,0.497504,00:01


Voy a ver que tal funciona

In [41]:
n,counts = 0,torch.zeros(len(vocab))
for x,y in dls.valid:
    n += y.shape[0]
    for i in range_of(vocab): counts[i] += (y==i).long().sum()
idx = torch.argmax(counts)
idx, vocab[idx.item()], counts[idx].item()/n

(tensor(1), '.', 0.15188970763014023)

**Red recurrente**

In [42]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [43]:
learn = Learner(dls, LMModel2(len(vocab), 64), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.711986,1.963469,0.463751,00:02
1,1.339630,1.774258,0.473497,00:01
2,1.397991,1.513624,0.493939,00:01
3,1.341037,1.513541,0.498455,00:01


**Mejorando la RNN**

Este es el tercero de mis modelos:

In [44]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

In [45]:
m = len(seqs)//bs
m,bs,len(seqs)

(328, 64, 21031)

In [46]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [47]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

In [48]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy, metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.534806,1.698777,0.525721,00:02
1,0.838410,1.527942,0.625721,00:01
2,0.664397,1.117358,0.705769,00:01
3,0.573060,1.017530,0.731731,00:01
4,0.559307,1.200025,0.755048,00:01
5,0.545655,1.158849,0.717308,00:01
6,0.482094,1.220648,0.731490,00:02
7,0.459266,1.175575,0.759375,00:02
8,0.410128,1.268622,0.746635,00:01
9,0.395750,1.309121,0.743990,00:01


Voy a crear más señal:

In [55]:
ls = 16
seqs = L((tensor(nums[i:i+ls]), tensor(nums[i-1:i+ls-1]))
         for i in range(0,len(nums)-ls-1,ls))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [56]:
[L(vocab[o] for o in s) for s in seqs[0]]

[(#16) ['one','.','two','.','three','.','four','.','five','.'...], (#0) []]

In [57]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(ls):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [58]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [59]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

**RNNs multicapa**

In [60]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [61]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time


RuntimeError: ignored

**LSTM**

In [ ]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(2, bs, n_hidden) for _ in range(n_layers)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.029677,2.723996,0.300618,00:01
1,2.193921,1.889679,0.246175,00:01
2,1.641660,1.807727,0.480143,00:01
3,1.371178,2.031465,0.516195,00:01
4,1.183990,1.970055,0.562419,00:01
5,0.959340,1.975672,0.607422,00:01
6,0.699868,1.815538,0.639079,00:01
7,0.480542,1.813820,0.678548,00:01
8,0.346177,1.701169,0.707520,00:01
9,0.249710,1.598971,0.753418,00:01


**Reduciendo el sobreajuste en LSTM**

In [ ]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [ ]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,2.466335,1.915134,0.481201,00:01
1,1.539552,1.287652,0.650146,00:01
2,0.813102,1.054421,0.722819,00:01
3,0.405256,0.850222,0.783040,00:01
4,0.214871,0.768760,0.812663,00:01
5,0.118563,0.743559,0.828857,00:01
6,0.072849,0.823633,0.814209,00:01
7,0.051636,0.689776,0.844808,00:01
8,0.037206,0.582271,0.855957,00:01
9,0.028060,0.742132,0.836100,00:01


Al finalizar recuerda guarda el notebook en tu repositorio de GitHub con la opción *Save in GitHub* del menú *File*.